In [1]:
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from sharadar.pipeline.factors import (
    MarketCap,
    EV,
    Fundamentals,
    LogFundamentalsTrend,
    LogTimeTrend,
    InvestmentToAssets
)
from sharadar.pipeline.engine import load_sharadar_bundle, symbol, symbols, make_pipeline_engine
from zipline.pipeline.filters import StaticAssets
from zipline.pipeline.factors import CustomFactor
import numpy as np

#pd.options.display.float_format = '{:.0f}'.format

In [3]:
class EarningYield(CustomFactor):
    inputs = [
        Fundamentals(field='netinccmnusd_art'),
        MarketCap()
    ]
    window_safe = True
    window_length = 1

    def compute(self, today, assets, out, earnings, mkt_cap):
        l = self.window_length
        out[:] = earnings[-l] / mkt_cap[-l]


month_length = 21
monthly = tuple(np.append(np.arange(-11 * month_length, 0, step=month_length), -1))

#universe = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
universe = StaticAssets(symbols(['IBM']))
pipe = Pipeline(columns={
    #'revenue': Fundamentals(field='revenue', window_length=1),
    #'revenue_trend': LogFundamentalsTrend(field='revenue', mask=universe).trend,
    'ey': EarningYield(),
    'ia': InvestmentToAssets()
    #'ey_trend': LogTimeTrend([EarningYield()], periodic=monthly).trend,
},
screen = universe
)

engine = make_pipeline_engine()
pipe_start = pd.to_datetime('2016-08-26', utc=True)
pipe_end = pd.to_datetime('2017-09-07', utc=True)
stocks = engine.run_pipeline(pipe, pipe_start, pipe_end)
stocks

[2020-06-10 19:48:28.283405] INFO: sharadar_db_bundle: Compute pipeline values in chunks of 120 days.
[2020-06-10 19:48:28.284260] INFO: sharadar_db_bundle: Compute values for pipeline from 2016-08-26 to 2017-02-16.
[2020-06-10 19:48:28.302310] INFO: sharadar_db_bundle: Computing term 1 of 9 [AssetExists()]
[2020-06-10 19:48:28.303672] INFO: sharadar_db_bundle: Term already in workspace: no computation needed
[2020-06-10 19:48:28.304522] INFO: sharadar_db_bundle: Computing term 2 of 9 [InvestmentToAssets([], 1)]


KeyError: 0